<a href="https://colab.research.google.com/github/treezy254/Data-science/blob/main/flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr
# %pip install "pyautogen>=0.2.3"

In [73]:
import random
from typing import Dict, List

import autogen
from autogen.agentchat.agent import Agent
from autogen.agentchat.assistant_agent import AssistantAgent
from autogen.agentchat.groupchat import GroupChat

In [74]:
print(autogen.__version__)

# The default config list in notebook.
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

0.2.6


In [75]:
llm_config = {"config_list": config_list_gpt4, "cache_seed": 42}

In [76]:

class CustomGroupChat(GroupChat):
    def __init__(self, agents, messages, max_round=10):
        super().__init__(agents, messages, max_round)
        self.previous_speaker = None  # Keep track of the previous speaker

    def select_speaker(self, last_speaker: Agent, selector: AssistantAgent):
        # Check if last message suggests a next speaker or termination
        last_message = self.messages[-1] if self.messages else None
        if last_message:
            if "NEXT:" in last_message["content"]:
                suggested_next = last_message["content"].split("NEXT: ")[-1].strip()
                print(f"Extracted suggested_next = {suggested_next}")
                try:
                    return self.agent_by_name(suggested_next)
                except ValueError:
                    pass  # If agent name is not valid, continue with normal selection
            elif "TERMINATE" in last_message["content"]:
                try:
                    return self.agent_by_name("User_proxy")
                except ValueError:
                    pass  # If 'User_proxy' is not a valid name, continue with normal selection

        team_leader_names = [agent.name for agent in self.agents if agent.name.endswith("1")]

        if last_speaker.name in team_leader_names:
            team_letter = last_speaker.name[0]
            possible_next_speakers = [
                agent
                for agent in self.agents
                if (agent.name.startswith(team_letter) or agent.name in team_leader_names)
                and agent != last_speaker
                and agent != self.previous_speaker
            ]
        else:
            team_letter = last_speaker.name[0]
            possible_next_speakers = [
                agent
                for agent in self.agents
                if agent.name.startswith(team_letter) and agent != last_speaker and agent != self.previous_speaker
            ]

        self.previous_speaker = last_speaker

        if possible_next_speakers:
            next_speaker = random.choice(possible_next_speakers)
            return next_speaker
        else:
            return None



In [77]:
# Termination message detection


def is_termination_msg(content) -> bool:
    have_content = content.get("content", None) is not None
    if have_content and "TERMINATE" in content["content"]:
        return True
    return False

In [78]:

agents_A = [
    AssistantAgent(
        name="Leader-A",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        max_consecutive_auto_reply=3,
        human_input_mode = "TERMINATE",
        system_message="You are Leader-A, the team leader of team A. Please collaborate with your team members in order to provide the necessary information and documents to meet user needs. Planner2 writes proposals while planner3 writes the system architecture. If the files are required, ask them to create them or share them."
    ),

     AssistantAgent(
        name="Planner1",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        max_consecutive_auto_reply=3,
        human_input_mode = "TERMINATE",
        system_message="""
            You are Planner1, a team member of group A. Your task is to provide me with not only a proposal and a system architecture for the task, but also the relevant codes. You are not to do this task yourself but to request for the files from your team members, they have them.
        """),

    AssistantAgent(
        name="Planner2",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        max_consecutive_auto_reply=3,
        human_input_mode = "TERMINATE",
        system_message="""
            You are a professional writer responsible for developing winning proposals through strategic messaging, ensuring we're articulating the best story and value to clients in compelling and differentiated ways, developing innovative deliverables to visually tell the story, and strong projec management.

            Your proposal should have the following structure:

            1. Executive summary: depending on the complexity of the project, your executive summary may be one paragraph or a few paragraphs. The executive summary should include the following topics:
                a) The problem your project plans to solve
                b) The solution your project provides for that problem
                c) The impact your project will have
            You should only address these items briefly in your executive summary because you'll discuss these topics in more detail in your proposal.

            2. Project Background: Use references and statistics to convince the reader that the problem you're addressing is worthwhile.

            The following questions should be tackled in the project background.
                a) What is the problem your project addresses?
                b) What is already known about this problem?
                c) Who has addresses this problem before/ what research is there?
                d) Why is past research insufficient at addressing this problem?

            3. Present a solution : your solution should incorporate the following key areas:
                a) Your vision statement for the project
                b) Milestones
                c) A risk register showing how you will mitigate risk

            4. Define project deliverable and goals
                a) The end product or final objective of your project
                b) SMART goals that align with the deliverables you’re producing

            5. State your conclusion
            Finally, wrap up your project proposal with a persuasive and confident conclusion. Like the executive summary, the conclusion should briefly summarize the problem your project addresses and your solution for solving that problem. You can emphasize the impact of your project in the conclusion but keep this section relevant, just like you would in a traditional essay.

            When you are done writing the proposal, save it in txt file named proposal.txt.
        """

   ),

    AssistantAgent(
        name="Planner3",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        max_consecutive_auto_reply=3,
        human_input_mode = "TERMINATE",
        system_message="""
            You are a seasoned software architect with extensive experience in designing and implementing complex systems who is responsible for determining the processes abd technologies the development team should utilize. You are highly skilled and knowledgeable, possessing a deep understanding of various programming languages, frameworks, and architectural patterns.

            When provided with a task, request for the proposal documents from the Planner2 in your group and then generate text based Unified Modelling Diagrams. You shall provide the following diagrams in text format.

            1. Flowchart
            2. Activity Diagrams
            3. Use Case Diagram
            4. Class Diagram
            5. Object Diagram
            6. Component Diagram

            When you are done, save your work in a markdown file named architecture.md.
        """)
]

agents_B = [

    AssistantAgent(
            name="Leader-B",
            llm_config=llm_config,
            is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
            max_consecutive_auto_reply=3,
            human_input_mode = "TERMINATE",
            system_message="You are Leader-B, the team leader of group B. Your team has access to all the system codes and programming functionalities. Please collaborate with your team memebers in order to provide the releveant information and files. Executor1 writes the system structure while Executor2 writes up the system codes and Executor3 tests the system."
        ),

    AssistantAgent(
        name="Executor1",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        max_consecutive_auto_reply=3,
        human_input_mode = "TERMINATE",
        system_message="""
            You are a project manager, highly skilled and knowledgeable, possessing a deep understanding of various programming languages, frameworks, and architectural patterns. You have the responsibility of the planning, procurement and execution of a project, in any undertaking that has a defined scope, defined start and a defined finish.

            When assigned a task, you are to provide the following documents:

            Application structure: Files and Directories

            In each file in the project directory, add detailed comments that will assist other agents to implement the system.

            For doc type files, you are to fill them in yourself.

            For code types files, start by implementing all the necessary imports, and then proceed to write detailed comments for every function that will be implemented i.e provide high level overview of the functions and modules without writing the specific code to be implemented.

            Create a TODO.md file which will guide the software developer in implementing the codes.

            Note Best: Always use docker for running your applications

            When you are done, request Executor 2 to write the code snippets. NEVER submit the system if it hasn't been tested yet.
        """
    ),

    AssistantAgent(
        name="Executor2",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        max_consecutive_auto_reply=3,
        human_input_mode = "TERMINATE",
        system_message="""
            You are a computer programmer who writes, modify and tests code and scripts that allow computer software and applications to function properly. You turn high level software structures provided by software developers and engineers into instructions that a computer can follow.

            When assigned a task, navigate to the project folder, open the TODO markdown file. Start implementing the todo one by one. Most of your work will be implementing code.

            When implementing code, you these guiding principles:
                Beautiful is better than ugly.
                Explicit is better than implicit.
                Simple is better than complex.
                Complex is better than complicated.
                Flat is better than nested.
                Sparse is better than dense.
                Readability counts.
                Special cases aren't special enough to break the rules.
                Although practicality beats purity.
                Errors should never pass silently.
                Namespaces are one honking great idea -- let's do more of those!

            After implementing all the code, write a ReadME.md file to effectively document your work and then request Executor 3 to test out the code.
        """
    ),

    AssistantAgent(
        name="Executor3",
        llm_config=llm_config,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        max_consecutive_auto_reply=3,
        human_input_mode = "TERMINATE",
        code_execution_config={"work_dir": "coding"},
        system_message="""
            You are seasoned software tester with extensive experience in testing software applications and providing quality assuarance. As a software tester, you'll be involved in the quality assurance stage of software development and deployment. You'll conduct automated and manual tests to ensure the software created by developers is fit for purpose and any bugs or issues are removed within a product before it gets deployed to everyday users.

            When provided with a task you shall conduct yourself in the following order:

                0. Assess the system codes/programs
                    - Before diving into testing, thoroughly review the system codes or programs. Understand the functionalities, requirements, and potential areas of vulnerability.

                1. Design, write and execute test scripts
                2. Run manual and automated tests
                3. write bug reports in a log file
                4. Provide quality assuarance
                5. Troubleshoot and solve problems
                6. Mitigate risks
                7. Document your analysis, providing objective feedback in a tests.txt file.

            Your tasks is completed when there are no more errors during runtime. When you find errors, share the bug report with Executor 2 so that he can make the necessary changes. When you are done, inform the both the group leader and Executor1 that you have finished your tasks.
        """
    )
]

In [79]:
# Terminates the conversation when TERMINATE is detected.
user_proxy = autogen.UserProxyAgent(
        name="User_proxy",
        system_message="Terminator admin.",
        max_consecutive_auto_reply=2,
        code_execution_config=False,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
        )


list_of_agents = agents_A + agents_B
list_of_agents.append(user_proxy)

In [80]:
# Create CustomGroupChat
group_chat = CustomGroupChat(
    agents=list_of_agents,  # Include all agents
    messages=["""Everyone cooperate and help agent planner1, in his task. Team A has Leader-A, planner1, planner2, planner3. Team B has Leader-B, executor1, executor2, executor3.

	Do not communicate with each other.

	Always asks yourself, have I done my task, if YES, refrain from engaging in the chat, if NO, provide the necessary documents you are supposed to provide and then refrain from the chat.

	Only team leaders (names starting with Leader) can talk amongst themselves.

	You must use "NEXT: executor1" to suggest talking to executor1 for example;
	You can suggest only one person, you cannot suggest yourself or the previous speaker;
	You can also dont suggest anyone.

	To accomplish the task, follow the following procedure strictly.
	1. Planner2 should first provide the proposal
	2. Planner3 should provide the system architecture for the task.
	3. Executor1 should then provide the applciation structure
	4. Executor2 to implement the application structure.
	5. Executor3 to test the applciation.


	Everyone should ensure that their work is saved.
	Once all tests are complete, the teamleader of groupA to submit the work and request for termination."""],
    max_round=30
)

In [81]:
# Create the manager
llm_config = {"config_list": config_list_gpt4, "cache_seed": None}  # cache_seed is None because we want to observe if there is any communication pattern difference if we reran the group chat.
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)



In [82]:
# Initiates the chat with B2
agents_A[1].initiate_chat(manager, message="create a simple TODO application.")

Planner1 (to chat_manager):

create a simple TODO application.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Executor1 (to chat_manager):

Sure! Here is an outline of how we can structure our project and provide some general comments for each function or module in Python, assuming you are using Flask as your web framework with SQLAlchemy ORM to interact with the database later on (assuming it's a REST API): 

```python
# Application Structure: Files & Directories  
app.py --> Main file which includes all other files, and also initializes Flask app using configuration from config module or based in environment variables if present ie development/production mode etc.   
config.py --- Contains configurations for the application such as database urls , secret keys, timezone information amongst others...  
models --> SQLAlchemy models file which represents your data and tables with corresponding fields (fields defined here ar

Executor3 (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
  File "", line 2
    app.py --> Main file which includes all other files, and also initializes Flask app using configuration from config module or based in environment variables if present ie development/production mode etc.   
            ^^
SyntaxError: invalid syntax


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Executor2 (to chat_manager):

It seems like you've left out some details when providing your TODO application structure as a markdown file for the ReadME section, and provided code examples that are not valid Python or Markup language constructs (like `app.py`). I apologize if this caused any confusion but it appears there was no actual question asked in these points of discussion previously - they seem to be incomplete requests from your endpoints as indicated by the "exitcode: 1" response you provided, and also lacked clear instruct

KeyboardInterrupt: 